In [161]:
import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt
%matplotlib inline
import warnings 
import random
import torch.nn as nn

warnings.filterwarnings('ignore')
torch.manual_seed(220);

In [226]:
batch_size = 32 # how many independent sequences we will process in parallel
block_size = 8 # what is the maximum context length for predictions
max_iters = 3000
eval_interval = 300
learning_rate = 1e-5
device = 'cuda' if torch.cuda.is_available() else 'cpu'
eval_iters = 200 
batch_size = 32

In [163]:
with open('names.txt', 'r') as f:
    words = f.read().splitlines()

words[:8]

['emma', 'olivia', 'ava', 'isabella', 'sophia', 'charlotte', 'mia', 'amelia']

In [164]:
# encoding letters to integers
charset = sorted(set(''.join(words)))
stoi = {s:i+1 for i,s in enumerate(charset)}
stoi['.'] = 0
vocab_size = len(stoi)
itos = {i:s for s,i in stoi.items()}
print(itos)
print(vocab_size)

{1: 'a', 2: 'b', 3: 'c', 4: 'd', 5: 'e', 6: 'f', 7: 'g', 8: 'h', 9: 'i', 10: 'j', 11: 'k', 12: 'l', 13: 'm', 14: 'n', 15: 'o', 16: 'p', 17: 'q', 18: 'r', 19: 's', 20: 't', 21: 'u', 22: 'v', 23: 'w', 24: 'x', 25: 'y', 26: 'z', 0: '.'}
27


In [165]:
# Creating the dataset 

def build_dataset(words):
    X = []
    Y = []
    for w in words:
        context = [0]*block_size 
        # print(w)
        for char in w + '.':
            X.append(context)
            Y.append(stoi[char])
            # print(f'{[itos[i] for i in context]} --> {char}')
            context = context[1:] + [stoi[char]]
    
    X = torch.tensor(X)
    Y = torch.tensor(Y)
    return X, Y 

random.seed(42)
random.shuffle(words)
n1 = int(0.8*len(words))
n2 = int(0.9*len(words))

Xtr, Ytr = build_dataset(words[:n1])
XDev, YDev = build_dataset(words[n1:n2])
Xts, Yts  = build_dataset(words[n2:])

In [166]:
def get_batch(split):
    X, Y  = {'train': (Xtr, Ytr),
            'val' : (XDev, YDev)}[split]
    ix = torch.randint(0, X.shape[0], (batch_size,))
    xb, yb = X[ix].to(device), Y[ix].to(device)
    return xb, yb

In [167]:
@torch.no_grad()
def split_loss(split):
    x, y = {'train': (Xtr, Ytr),
            'dev' : (XDev, YDev),
            'test' : (Xts, Yts)}[split]
    logits = model(x)
    loss = F.cross_entropy(logits, y)
    print(split, loss.item())
    
# split_loss('train')
# split_loss('dev')

In [168]:
class Head(nn.Module):
    def __init__(self, head_size):
        super().__init__()
        self.key = nn.Linear(vocab_size, head_size, bias=False)
        self.query = nn.Linear(vocab_size, head_size, bias=False)
        self.value = nn.Linear(vocab_size, head_size, bias=False)
        self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))
        
    def forward(self, x):
        B,T,C = x.shape
        k = self.key(x) # (B,T,C)
        q = self.query(x) # (B,T,C)
        
        # compute attention scores ('affinities')        
        wei = q @ k.transpose(-2, -1) * C**-0.5 # (B,T,C) @ (B,C,T) --> (B,T,T)
        wei = wei.masked_fill(self.tril[:T,:T] == 0, float('-inf'))
        wei = F.softmax(wei, dim=-1) # (B,T,T)
        # perform the weighted aggregation of values 
        v = self.value(x)
        out = wei @ v # (B,T,T) @ (B,T,C) --> (B,T,C)
        return out 

In [184]:
class BigramLngModel(nn.Module):

    def __init__(self, vocab_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)
        self.pos_embedding_table = nn.Embedding(block_size, vocab_size)
        self.sa = Head(16)
        self.linear = nn.Linear(16*block_size, vocab_size)
        self.tanh = nn.Tanh()
        
    def forward(self, x, targets=None):
        B, T = x.shape
        tok = self.token_embedding_table(x) # B, T, C / 32, 3, 27
        pos = self.pos_embedding_table(torch.arange(T, device=device))
        x = tok + pos 
        x = self.sa(x)
        x = x.view(B, -1)
        x = torch.tanh(x)
        logits = self.linear(x)
        if targets is None:
            loss = None
        else:
            loss = F.cross_entropy(logits, targets)
        return logits, loss

    def generate(self, names):
        # idx is (B, T) array of indices in the current context 
        for _ in range(names):
            out = []
            context = [0]*block_size
            while True:
                logits,_ = model(torch.tensor([context]).to(device))
                probs = F.softmax(logits, dim=1)
                ix = torch.multinomial(probs, num_samples=1).item()
                context = context[1:] + [ix]
                out.append(ix)
                if ix == 0:
                    break
        
            print(''.join(itos[i] for i in out))

In [185]:
model = BigramLngModel(vocab_size)
m = model.to(device)

# create a pytorch optimizer


In [227]:
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)


In [228]:
lossi = []

In [ ]:
for iter in range(max_iters):
    
    # every once in a while evaluate the loss on train and val sets 
    # if iter %  eval_interval == 0:
    # if iter %  eval_interval == 
    #     loss = estimate_loss()
    #     print(f"step {iter}: train loss {loss['train']:.4f}, val loss {loss['val']:.4f}")
    
    # sample a batch of training examples
    xb, yb = get_batch('train')
    
    # evaluate the loss 
    logits, loss = model(xb,yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step() 
    if iter %  eval_interval == 0:
        print(f'{loss.item():.4f}')
    lossi.append(loss.item())

2.2267
2.0966
2.0086
2.3684
1.9308
2.3239
2.1731
2.6377
2.0369


In [ ]:
plt.plot(torch.tensor(lossi).view(-1, 100).mean(dim=1))

In [ ]:
m.generate(10)